<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/HOML_Exercise_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


In [2]:
np.random.seed(42)
tf.random.set_seed(42)

#8.

# Grammer_StringGenerater

In [3]:
#문법에 맞는 문자열을 반환하는 함수

#문법, LR table 과 비슷함
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    #마지막
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    #마지막
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output=[]
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        #다음 문자와 상태
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [4]:
for _ in range(25):
    #25개의 문자열 생성
    print(generate_string(default_reber_grammar), end=" ")

BTXXTTVPXTVPXTTVPSE BPVPSE BTXSE BPVVE BPVVE BTSXSE BPTVPXTTTVVE BPVVE BTXSE BTXXVPSE BPTTTTTTTTVVE BTXSE BPVPSE BTXSE BPTVPSE BTXXTVPSE BPVVE BPVVE BPVVE BPTTVVE BPVVE BPVVE BTXXVVE BTXXVVE BTXXVPXVVE 

#Wrong_StringGenrater

In [5]:
#모든 가능한 알파벳들
POSSIBLE_CHARS = "BEPSTVX"

#잘못된 문자열 만들기
def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    #올바른 문자열에
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    #있을수는 있지만 올바른 문자열엔 없는 문자열을
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    #중간에 하나 추가함
    return good_string[:index] + bad_char + good_string[index + 1:]

In [6]:
#잘못된 문자열
for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

BTBPVVETV BTBTSSSPXXVVETE BTBTXXTTSTVPSETE BBBTXSETE BPBTPXSEPE BPBPVPXTTTTVPXTVPXVPXTTTVVEVE BTBXXXTVPSETE BEBTSSSSSXXVPXTVVETE BTBXTTVVETE BPBTXSTPE BTBTXXTTTVPSBTE BTBTXSETX BTBTSXSSTE BPBPVVEPT BTBPTVEETE BTBTSSXXTTVXETE BTBTSXTTVVETE BPBPVVTPE BTBTSXTTVVETE EPBPVPXVVEPE BPTTXSEPE BPBTXXSPXTVVEPE BTBTXSPTE BPTTSXXTVPXVVEPE PPBPVPSEPE 

#Char_To_Int

In [7]:
#문자열을 숫자로 바꾸기, 문자의 인덱스 리스트로 변경한다.
def strings_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

#DataGenerater

In [8]:
#올바른 문자열, 잘못된 문자열 1 : 1 인 샘플과 레이블 생성
def generate_dataset(size):
    good_strings = [strings_to_ids(generate_string(embedded_reber_grammar)) for _ in range(size // 2)]
    bad_strings = [strings_to_ids(generate_corrupted_string(embedded_reber_grammar)) for _ in range(size // 2)]
    all_strings = good_strings + bad_strings
    #ragged 텐서 생성
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] + [[0.] for _ in range(len(bad_strings))])
    return X, y


#Generate_Data

In [9]:
X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)
X_test, y_test = generate_dataset(2000)

In [10]:
print(X_train[0])
s = ''
for i in X_train[0]:
    s += POSSIBLE_CHARS[i]
print(s, y_train[0][0])

tf.Tensor([0 4 0 2 5 5 1 4 1], shape=(9,), dtype=int32)
BTBPVVETE 1.0


#Binary_Sequence_Classifier

In [11]:
embedding_size = 5

model = keras.models.Sequential([
    #인풋데이터는 ragged tensor, int32 타입
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation='sigmoid')
])

#네스테로프 가속 경사하강법 
optimizer = keras.optimizers.SGD(learning_rate=0.02, momentum=0.95, nesterov=True)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#Model_Train

In [12]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/gru/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/gru/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 5), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/gru/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


313/313 [==============================] - 33s 92ms/step - loss: 0.6910 - accuracy: 0.5093 - val_loss: 0.6825 - val_accuracy: 0.5595
Epoch 2/20
313/313 [==============================] - 30s 96ms/step - loss: 0.6678 - accuracy: 0.5720 - val_loss: 0.6706 - val_accuracy: 0.5975
Epoch 3/20
313/313 [==============================] - 31s 100ms/step - loss: 0.6505 - accuracy: 0.5796 - val_loss: 0.6472 - val_accuracy: 0.6135
Epoch 4/20
313/313 [==============================] - 31s 98ms/step - loss: 0.6356 - accuracy: 0.5963 - val_loss: 0.6239 - val_accuracy: 0.6290
Epoch 5/20
313/313 [==============================] - 30s 96ms/step - loss: 0.6059 - accuracy: 0.6342 - val_loss: 0.5778 - val_accuracy: 0.7010
Epoch 6/20
313/313 [==============================] - 30s 96ms/step - loss: 0.5474 - accuracy: 0.7015 - val_loss: 0.5608 - val_accuracy: 0.5500
Epoch 7/20
313/313 [==============================] - 31s 98ms/step - loss: 0.4107 - accuracy: 0.8141 - val_loss: 0.4382 - val_accuracy: 0.7820
Ep

#Model_Evaluate

In [13]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 1s 15ms/step - loss: 2.1198e-04 - accuracy: 1.0000


[0.00021197657042648643, 1.0]

#9.

#9-1

#Random_Date_Genrater

In [14]:
from datetime import date

#January 22, 2019 -> 2019-01-22 형식으로
MONTHS = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    #숫자
    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    #month, day, year 을 빼내기위해서
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    #샘플
    X = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    #타깃 YYYY-mm-dd 형식으로 변환
    y = [dt.isoformat() for dt in dates]
    
    return X, y

In [15]:
n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
July 22, 1344            1344-07-22               
March 21, 7185           7185-03-21               
January 02, 5192         5192-01-02               


#INPUT_OUTPUT_CHARS

In [16]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
OUTPUT_CHARS = "0123456789-"
INPUT_CHARS, OUTPUT_CHARS

(' ,0123456789ADFJMNOSabceghilmnoprstuvy', '0123456789-')

#Input_Output_Tokenizer

In [18]:
input_chars = MONTHS + ['0123456789, ']
output_chars = ['0123456789-']

input_tokenizer = keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
input_tokenizer.fit_on_texts(sorted(set(input_chars)))
input_vocab_size = len(input_tokenizer.word_index)

output_tokenizer = keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
output_tokenizer.fit_on_texts(sorted(set(output_chars)))
output_vocab_size = len(output_tokenizer.word_index)

In [19]:
print(input_tokenizer.word_index)
print(output_tokenizer.word_index)

{'e': 1, 'r': 2, 'u': 3, 'b': 4, 'a': 5, 'y': 6, 't': 7, 'c': 8, 'm': 9, 'J': 10, 'A': 11, 'p': 12, 'l': 13, 'n': 14, 'M': 15, 'o': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ',': 27, ' ': 28, 'i': 29, 'g': 30, 's': 31, 'D': 32, 'F': 33, 'h': 34, 'N': 35, 'v': 36, 'O': 37, 'S': 38}
{'0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, '-': 11}


In [20]:
X, y = random_dates(3)

print('-'*25)
print(X[0])
X = input_tokenizer.texts_to_sequences(X)
print(X[0])
X = input_tokenizer.sequences_to_texts(X)
print(X[0][::2])

print('-'*25)

print(y[0])
y = output_tokenizer.texts_to_sequences(y)
print(y[0])
y = output_tokenizer.sequences_to_texts(y)
print(y[0][::2])


-------------------------
April 09, 3947
[11, 12, 2, 29, 13, 28, 17, 26, 27, 28, 20, 26, 21, 24]
April 09, 3947
-------------------------
3947-04-09
[4, 10, 5, 8, 11, 1, 5, 11, 1, 10]
3947-04-09


#Dataset_Generater

In [21]:
def prepare_date_strs(date_strs, tokenizer):
    ids = [dt for dt in tokenizer.texts_to_sequences(date_strs)]
    X = tf.ragged.constant(ids, ragged_rank=1)
    return X.to_tensor() #ragged 텐서를 기본 텐서로 부족한 부분은 0 패딩토큰으로 채운다.

def create_dataset(n_dates):
    X, Y = random_dates(n_dates)
    return prepare_date_strs(X, input_tokenizer), prepare_date_strs(Y, output_tokenizer)

In [22]:
X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [23]:
print(X_train[0], Y_train[0], sep='\n')
print(input_tokenizer.sequences_to_texts([X_train[0].numpy()])[0][::2])
print(output_tokenizer.sequences_to_texts([Y_train[0].numpy()])[0][::2])

tf.Tensor([15  5  6 28 17 22 27 28 22 24 18 24  0  0  0  0  0  0], shape=(18,), dtype=int32)
tf.Tensor([ 6  8  2  8 11  1  6 11  1  6], shape=(10,), dtype=int32)
May 05, 5717
5717-05-05


#Seq2Seq_Model

In [24]:
embedding_size = 32
#출력 최대 길이 : 10 (yyyy-mm-dd)
max_output_length = Y_train.shape[-1]

#인코더 디코더 구조를 사용한다.

#인코더, 시퀀스 투 벡터 구조
encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=input_vocab_size + 1, output_dim=embedding_size, input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(output_vocab_size + 1, activation='softmax')
])

model = keras.models.Sequential([
    encoder,
    #출력길이가 10이니깐 인코더 출력을 10번 반복해서, 디코더가 10개의 출력을 만들도록함
    keras.layers.RepeatVector(max_output_length),
    decoder
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 128)               83680     
                                                                 
 repeat_vector (RepeatVector  (None, 10, 128)          0         
 )                                                               
                                                                 
 sequential_2 (Sequential)   (None, 10, 12)            133132    
                                                                 
Total params: 216,812
Trainable params: 216,812
Non-trainable params: 0
_________________________________________________________________


In [25]:
optimizer = keras.optimizers.Nadam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [26]:
history = model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid))

Epoch 1/20
313/313 [==============================] - 11s 15ms/step - loss: 1.8123 - accuracy: 0.3460 - val_loss: 1.4039 - val_accuracy: 0.4805
Epoch 2/20
313/313 [==============================] - 4s 14ms/step - loss: 1.2734 - accuracy: 0.5424 - val_loss: 1.0721 - val_accuracy: 0.6217
Epoch 3/20
313/313 [==============================] - 4s 14ms/step - loss: 0.9944 - accuracy: 0.6487 - val_loss: 0.8298 - val_accuracy: 0.7014
Epoch 4/20
313/313 [==============================] - 4s 13ms/step - loss: 0.7202 - accuracy: 0.7332 - val_loss: 0.6222 - val_accuracy: 0.7632
Epoch 5/20
313/313 [==============================] - 4s 13ms/step - loss: 0.7677 - accuracy: 0.7216 - val_loss: 0.5274 - val_accuracy: 0.7948
Epoch 6/20
313/313 [==============================] - 4s 13ms/step - loss: 0.5844 - accuracy: 0.7871 - val_loss: 0.4153 - val_accuracy: 0.8364
Epoch 7/20
313/313 [==============================] - 4s 14ms/step - loss: 0.3369 - accuracy: 0.8690 - val_loss: 0.2816 - val_accuracy: 0.890

In [27]:
model.evaluate(X_test,Y_test)

63/63 [==============================] - 0s 6ms/step - loss: 0.0027 - accuracy: 1.0000


[0.0027159270830452442, 1.0]

#Model_Return_To_Strings

In [28]:
def ids_to_strings(ids, tokenizer):
    return tokenizer.sequences_to_texts(ids)

print('-'*25)
#X_new = X_test[:3]
#패딩이 없는 데이터셋이라 잘 예측을 못함
X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"], input_tokenizer)
X_samples = ids_to_strings(X_new.numpy(), input_tokenizer)
print(*[x_sample[::2] for x_sample in X_samples], sep='\n')


print('-'*25)
Y_new = np.argmax(model.predict(X_new), axis=-1)
Y_new = ids_to_strings(Y_new, output_tokenizer)
print(*[y_sample[::2] for y_sample in Y_new], sep='\n')

-------------------------
May 02, 2020
July 14, 1789
-------------------------
2020-02-02
1789-09-14


In [29]:
max_input_length = X_train.shape[1]

def prepare_date_strs_with_padded(date_strs, tokenizer):
    X = prepare_date_strs(date_strs, tokenizer)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, paddings=[[0,0], [0, max_input_length - X.shape[1]]])
    return X


print('-'*25)
#패딩추가, 잘예측함
X_new = prepare_date_strs_with_padded(["May 02, 2020", "July 14, 1789"], input_tokenizer)
X_samples = ids_to_strings(X_new.numpy(), input_tokenizer)
print(*[x_sample[::2] for x_sample in X_samples], sep='\n')


print('-'*25)
Y_new = np.argmax(model.predict(X_new), axis=-1)
Y_new = ids_to_strings(Y_new, output_tokenizer)
print(*[y_sample[::2] for y_sample in Y_new], sep='\n')

-------------------------
May 02, 2020
July 14, 1789
-------------------------
2020-05-02
1789-07-14


#9-2

#Decoder_Input_Target_Shift

In [30]:
sos_id = output_vocab_size + 1

def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)

#타깃을 오른쪽으로 한번 shift 하고 맨앞에 sos 토큰을 추가한 디코더 입력
#원래는 인코더벡터를 복사해서 디코더에 입력했지만, 쉬프트된 타깃을 주입해서 이전타깃이 무엇인지 확인할수 있게한다.
X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

In [31]:
ids_to_strings(X_train_decoder[:5].numpy(), output_tokenizer)

['5 7 1 7 - 0 5 - 0',
 '3 7 6 2 - 0 6 - 1',
 '6 5 6 2 - 1 2 - 0',
 '3 9 8 7 - 0 3 - 0',
 '4 8 3 0 - 0 6 - 0']

#Model

In [32]:
encoder_embedding_size = 32
decoder_embedding_size = 32

lstm_units = 128

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = keras.layers.Embedding(input_dim=input_vocab_size+1, output_dim=encoder_embedding_size)(encoder_input)
#인코더의 출력벡터는 무시한다!!
_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = keras.layers.Embedding(output_vocab_size+1, decoder_embedding_size)(decoder_input)
#대신 인코더의 LSTM의 상태와 디코더의 LSTM과 연결시킨다.
decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(decoder_embedding, initial_state=encoder_state)
decoder_output = keras.layers.Dense(output_vocab_size+1, activation='softmax')(decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 32)     1248        ['input_2[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 32)     384         ['input_3[0][0]']                
                                                                                              

In [33]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10, validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/10
313/313 [==============================] - 9s 16ms/step - loss: 1.6702 - accuracy: 0.3712 - val_loss: 1.4184 - val_accuracy: 0.4505
Epoch 2/10
313/313 [==============================] - 4s 13ms/step - loss: 1.2336 - accuracy: 0.5262 - val_loss: 0.9797 - val_accuracy: 0.6413
Epoch 3/10
313/313 [==============================] - 4s 13ms/step - loss: 0.6811 - accuracy: 0.7581 - val_loss: 0.4135 - val_accuracy: 0.8705
Epoch 4/10
313/313 [==============================] - 4s 13ms/step - loss: 0.2387 - accuracy: 0.9428 - val_loss: 0.1191 - val_accuracy: 0.9822
Epoch 5/10
313/313 [==============================] - 4s 12ms/step - loss: 0.0761 - accuracy: 0.9923 - val_loss: 0.0491 - val_accuracy: 0.9969
Epoch 6/10
313/313 [==============================] - 4s 14ms/step - loss: 0.0680 - accuracy: 0.9901 - val_loss: 0.0258 - val_accuracy: 0.9994
Epoch 7/10
313/313 [==============================] - 4s 13ms/step - loss: 0.0408 - accuracy: 0.9946 - val_loss: 0.0202 - val_accuracy: 0.9994

In [34]:
print(ids_to_strings(np.argmax(model.predict([X_train[:5], X_train_decoder[:5]]), axis=-1), output_tokenizer))
print(ids_to_strings(Y_train[:5].numpy(), output_tokenizer))

['5 7 1 7 - 0 5 - 0 5', '3 7 6 2 - 0 6 - 1 3', '6 5 6 2 - 1 2 - 0 1', '3 9 8 7 - 0 3 - 0 4', '4 8 3 0 - 0 6 - 0 5']
['5 7 1 7 - 0 5 - 0 5', '3 7 6 2 - 0 6 - 1 3', '6 5 6 2 - 1 2 - 0 1', '3 9 8 7 - 0 3 - 0 4', '4 8 3 0 - 0 6 - 0 5']


In [35]:
#X_new = output_tokenizer.texts_to_sequences(["1789-07-14", "2020-05-01"])

def predict_date_strs(date_strs):
    X = prepare_date_strs_with_padded(date_strs, input_tokenizer)
    #X = date_strs
    sos_id = output_vocab_size + 1
    Y_pred = tf.fill(dims=(len(X), 1), value=sos_id)

    for index in range(max_output_length):
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, max_output_length-index]])
        Y_probas = tf.argmax(model.predict([X, X_decoder]), axis=-1, output_type=tf.int32)
        Y_pred = tf.concat([Y_pred, Y_probas[:, index:index+1]], axis=1)
    return ids_to_strings(Y_pred.numpy(), output_tokenizer)

print(predict_date_strs(["May 02, 2020", "July 14, 1789"]))

['2 0 2 0 - 0 5 - 0 2', '1 7 8 9 - 0 7 - 1 4']


#9-3

#Addon_Sampler_Model

In [41]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 12.4 MB/s 


In [46]:
import tensorflow_addons as tfa

encoder_embedding_size = 32
decoder_embedding_size = 32

units = 128

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

encoder_embeddings = keras.layers.Embedding(input_vocab_size + 1, encoder_embedding_size)(encoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True)
encoder_output, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]


decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)

decoder_embedding_layer = keras.layers.Embedding(output_vocab_size + 2, decoder_embedding_size)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

#sampler
sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(units)
output_layer = keras.layers.Dense(output_vocab_size+1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)

final_output, final_state, final_sequence_lengths = decoder(decoder_embeddings, initial_state=encoder_state)
Y_proba = keras.layers.Activation('softmax')(final_output.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[Y_proba])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [47]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=9, validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/15
313/313 [==============================] - 23s 60ms/step - loss: 1.6922 - accuracy: 0.3606 - val_loss: 1.4389 - val_accuracy: 0.4472
Epoch 2/15
313/313 [==============================] - 18s 59ms/step - loss: 1.2261 - accuracy: 0.5346 - val_loss: 0.9129 - val_accuracy: 0.6691
Epoch 3/15
313/313 [==============================] - 16s 52ms/step - loss: 0.6279 - accuracy: 0.7817 - val_loss: 0.3537 - val_accuracy: 0.8938
Epoch 4/15
313/313 [==============================] - 18s 58ms/step - loss: 0.2086 - accuracy: 0.9526 - val_loss: 0.0977 - val_accuracy: 0.9894
Epoch 5/15
313/313 [==============================] - 16s 52ms/step - loss: 0.0657 - accuracy: 0.9941 - val_loss: 0.0384 - val_accuracy: 0.9987
Epoch 6/15
313/313 [==============================] - 18s 58ms/step - loss: 0.0725 - accuracy: 0.9864 - val_loss: 0.1427 - val_accuracy: 0.9684
Epoch 7/15
313/313 [==============================] - 17s 54ms/step - loss: 0.0301 - accuracy: 0.9983 - val_loss: 0.0161 - val_accuracy:

In [48]:
print(predict_date_strs(["May 02, 2020", "July 14, 1789"]))

['2 0 2 0 - 0 5 - 0 2', '1 7 8 9 - 0 7 - 1 4']


#GreedyEmbeddingSampler

In [53]:
inference_sampler = tfa.seq2seq.sampler.GreedyEmbeddingSampler(embedding_fn=decoder_embedding_layer)

inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, inference_sampler, output_layer=output_layer, maximum_iterations=max_output_length)
batch_size = tf.shape(encoder_inputs)[:1]
start_tokens = tf.fill(dims=batch_size, value=sos_id)
final_outputs, final_state, final_sequence_lengths = inference_decoder(start_tokens, initial_state=encoder_state, start_tokens=start_tokens, end_token=0)

inference_model = keras.models.Model(inputs=[encoder_inputs], outputs=[final_outputs.sample_id])

In [66]:
def fast_predict_date_strs(date_strs):
    X = prepare_date_strs_with_padded(date_strs, input_tokenizer)
    Y_pred = inference_model.predict(X)
    return ids_to_strings(Y_pred, output_tokenizer)

In [55]:
%timeit predict_date_strs(["July 14, 1789", "May 01, 2020"])

1 loop, best of 5: 633 ms per loop


In [67]:
%timeit fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])

10 loops, best of 5: 68.1 ms per loop


#9-4

#Schedule_Model

In [70]:
n_epochs = 20
encoder_embedding_size = 32
decoder_embedding_size = 32
units = 128

#encoder
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

encoder_embeddings = keras.layers.Embedding(input_vocab_size + 1, encoder_embedding_size)(encoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True)
encoder_output, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

#decoder
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)\

decoder_embedding_layer = keras.layers.Embedding(output_vocab_size + 2, decoder_embedding_size)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

#sampler
#훈련할때 점진적으로 타깃에서 예측한 값을 입력으로 보낸다.
sampler = tfa.seq2seq.sampler.ScheduledEmbeddingTrainingSampler(sampling_probability=0., embedding_fn=decoder_embedding_layer)
sampler.sampling_probability = tf.Variable(0.)

decoder_cell = keras.layers.LSTMCell(units)
output_layer = keras.layers.Dense(output_vocab_size+1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)

final_output, final_state, final_sequence_lengths = decoder(decoder_embeddings, initial_state=encoder_state)
Y_proba = keras.layers.Activation('softmax')(final_output.rnn_output)

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

#Schedule_Sampling_Probability_Callback

In [71]:
#매 에폭이 끝날때마다 예측한 값을 입력으로 보낼 확률을 올려줌
def update_sampling_probability(epoch, logs):
    proba = min(1.0, epoch / (n_epochs - 10))
    sampler.sampling_probability.assign(proba)

sampling_probability_cb = keras.callbacks.LambdaCallback(on_epoch_begin=update_sampling_probability)

In [72]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=n_epochs, 
                    validation_data=([X_valid, X_valid_decoder], Y_valid), 
                    callbacks=[sampling_probability_cb])

Epoch 1/20
313/313 [==============================] - 22s 55ms/step - loss: 0.0666 - accuracy: 0.9853 - val_loss: 0.1372 - val_accuracy: 0.9571
Epoch 2/20
313/313 [==============================] - 17s 55ms/step - loss: 0.0072 - accuracy: 0.9993 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 3/20
313/313 [==============================] - 17s 53ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 4/20
313/313 [==============================] - 17s 55ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 5/20
313/313 [==============================] - 16s 52ms/step - loss: 8.2835e-04 - accuracy: 1.0000 - val_loss: 9.2663e-04 - val_accuracy: 1.0000
Epoch 6/20
313/313 [==============================] - 18s 56ms/step - loss: 6.6560e-04 - accuracy: 1.0000 - val_loss: 7.6463e-04 - val_accuracy: 1.0000
Epoch 7/20
313/313 [==============================] - 17s 53ms/step - loss: 5.5028e-04 - accuracy: 1.0000 - val_loss: 6.

#Inference_SampleEmbeddingSampler

In [73]:
softmax_temperature = tf.Variable(1.)

inference_sampler = tfa.seq2seq.sampler.SampleEmbeddingSampler(embedding_fn=decoder_embedding_layer, softmax_temperature=softmax_temperature)
inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, inference_sampler, output_layer=output_layer, maximum_iterations=max_output_length)

batch_size = tf.shape(encoder_inputs)[:1]
start_tokens = tf.fill(dims=batch_size, value=sos_id)

final_outputs, final_state, final_sequence_lengths = inference_decoder(start_tokens, initial_state=encoder_state, start_tokens=start_tokens, end_token=0)

inference_model = keras.models.Model(inputs=[encoder_inputs], outputs=[final_outputs.sample_id])

In [78]:
def creative_predict_date_strs(date_strs, temperature=1.0):
    softmax_temperature.assign(temperature)
    X = prepare_date_strs_with_padded(date_strs, input_tokenizer)
    Y_pred = inference_model.predict(X)
    return ids_to_strings(Y_pred, output_tokenizer)

print(creative_predict_date_strs(["July 14, 1789", "May 01, 2020"]))
print(creative_predict_date_strs(["July 14, 1789", "May 01, 2020"], 5.0))

['9 4 6 4 5 1 7 8 1 -', '9 3 8 9 7 3 2 5 7 7']
['6 6 6 - 0 1 4 - - 2', '4 6 9 3 3 4 1 - 9 0']
